In [1]:
import tensorflow as tf

In [2]:
import math

In [3]:
def arctanh(x):
    return tf.log(tf.divide(1+x,1-x))

Initialise all of the variables

In [4]:
def inner_prod(r_in, r_out, theta_in, theta_out):
    cosine = tf.cos(theta_in - theta_out)
    radius = tf.multiply(arctanh(r_in), arctanh(r_out))
    return 4 * tf.multiply(cosine, radius)

In [5]:
def minkowski_dot(u,v):
    return tf.tensordot(u,v,1) - 2*tf.multiply(u[0],v[0])

In [6]:
def exponential(base, tangent):
    """
    Compute the exponential of `tangent` from the point `base`.
    """
    #tangent = tangent.copy()
    norm = tf.sqrt(tf.maximum(minkowski_dot(tangent, tangent), 0))
    if norm == 0:
        return base
    tangent /= norm
    return tf.cosh(norm) * base + tf.sinh(norm) * tangent

In [7]:
def tensor_inner_prod(r_example, r_sample, theta_example, theta_sample):
    r1 = arctanh(r_example)
    r2 = arctanh(r_sample)
    radius_term = r1[:, None] + r2[None, :]
    cos_term = theta_example[:, None] - theta_sample[None, :]
    return tf.squeeze(4* tf.multiply(cos_term, radius_term))

In [8]:
def nce_loss(true_logits, sampled_logits):
        true_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.ones_like(true_logits), logits=true_logits)
        sampled_xent = tf.nn.sigmoid_cross_entropy_with_logits(
            labels=tf.zeros_like(sampled_logits), logits=sampled_logits)
        nce_loss_tensor = (tf.reduce_sum(true_xent) +
                           tf.reduce_sum(sampled_xent)) / 2
        return nce_loss_tensor

In [9]:
radius_in = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))  # radius
# radius_out = tf.Variable(tf.zeros([5,1]))
radius_out = tf.Variable(tf.sqrt(tf.random_uniform([5,1])))
theta_in = tf.Variable(2*np.pi*tf.random_uniform([5,1]))  # angle
theta_out = tf.Variable(2*np.pi*tf.random_uniform([5,1]))
# theta_out = tf.Variable(tf.zeros([5,1]))
sm_b = tf.Variable(tf.zeros([5,1]))
examples = tf.Variable([1.0,2.0])
labels = tf.Variable([3.0,4.0])

In [10]:
example_radius = tf.nn.embedding_lookup(radius_in, examples)
example_theta = tf.nn.embedding_lookup(theta_in, examples)
true_radius = tf.nn.embedding_lookup(radius_out, labels)
true_theta = tf.nn.embedding_lookup(theta_out, labels)

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64

In [11]:
sampled_ids = tf.Variable([0,1,2])
sampled_radius = tf.nn.embedding_lookup(radius_out, sampled_ids)
sampled_theta = tf.nn.embedding_lookup(theta_out, sampled_ids)
true_b = tf.nn.embedding_lookup(sm_b, labels)

TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: int32, int64

In [12]:
true_logits = inner_prod(example_radius, true_radius, example_theta, true_theta) + true_b

NameError: name 'example_radius' is not defined

In [13]:
sampled_b = tf.nn.embedding_lookup(sm_b, sampled_ids)
sampled_b_vec = tf.reshape(sampled_b, sampled_ids.get_shape())

In [14]:
sampled_logits = tensor_inner_prod(example_radius, sampled_radius, example_theta, sampled_theta) + sampled_b_vec

NameError: name 'example_radius' is not defined

In [15]:
r1 = arctanh(example_radius)
r2 = arctanh(sampled_radius)
radius_term = r1[:, None] + r2[None, :]
cos_term = example_theta[:, None] - sampled_theta[None, :]
retval = 4* tf.multiply(cos_term, radius_term)

NameError: name 'example_radius' is not defined

In [16]:
loss = nce_loss(true_logits, sampled_logits)

NameError: name 'true_logits' is not defined

In [17]:
def minkowski_dist(u, v):
    """
    The distance between two points in Minkowski space
    :param u:
    :param v:
    :return:
    """
    return tf.acosh(-minkowski_dot(u, v))

In [18]:
def project_onto_tangent_space(hyperboloid_point, minkowski_tangent):
    """
    project gradients in the ambiant space onto the tangent space
    :param hyperboloid_point:
    :param minkowski_tangent:
    :return:
    """
    return minkowski_tangent + minkowski_dot(hyperboloid_point, minkowski_tangent) * hyperboloid_point

In [28]:
# V = tf.Variable(np.array([[1,2],[4,5],[5,2]]), dtype=tf.int32)
V = tf.Variable(np.array([1,2]), dtype=tf.int32)
x = np.eye(V.shape[0])
x[0,0]=-1
T = tf.constant(x, dtype=tf.int32)
# U = tf.matmul(V,T)
U = tf.tensordot(V,T, 1)
# y = tf.constant([math.sqrt(10001),100])
y = tf.constant([1.,0])
z = tf.constant([1.,0])
w = tf.constant([0.,1.])
v = tf.constant([1.,1.])
s = tf.constant([math.sqrt(5), 2])
t = tf.constant([6.2, 3.2])

In [29]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
# print(sess.run(sampled_radius))
# print(sess.run(r1))
# print(sess.run(r2))
# print(sess.run(radius_term))
# print(sess.run(cos_term))
# print(sess.run(retval))
# print(sess.run(sampled_logits))
# print(sess.run(minkowski_dist(y,z)))
# print(sess.run(exponential(examples,labels)))
# print(sess.run(loss))
print(sess.run(minkowski_dot(y,z)))
print(sess.run(project_onto_tangent_space(y,z)))
print(sess.run(minkowski_dot(y,w)))
print(sess.run(project_onto_tangent_space(y,w)))
print(sess.run(minkowski_dot(y,v)))
print(sess.run(project_onto_tangent_space(y,v)))
temp1 = sess.run(project_onto_tangent_space(s,t))
temp2 = sess.run(project_onto_tangent_space(s,temp1))
print(temp1, temp2)


-1.0
[ 0.  0.]
0.0
[ 0.  1.]
-1.0
[ 0.  1.]
(array([-10.48916531, -11.72724247], dtype=float32), array([-10.48916531, -11.72724247], dtype=float32))


In [217]:
r1.get_shape()

TensorShape([Dimension(2), Dimension(1)])

In [224]:
tf.squeeze(example_radius).get_shape()

TensorShape([Dimension(2)])

In [219]:
retval.get_shape()

TensorShape([Dimension(2), Dimension(3), Dimension(1)])

In [229]:
sampled_logits.get_shape()

TensorShape([Dimension(2), Dimension(3)])

In [250]:
x1 = np.array([1,2])
x2 = np.array([3,4])
x3 = np.concatenate((x1.T,x2.T), axis=1)
x3

IndexError: axis 1 out of bounds [0, 1)